In [1]:
# 📌 03_predict.ipynb - Predict ETF forward returns with registered MLflow model

import pandas as pd
from pathlib import Path
from sklearn.metrics import root_mean_squared_error, r2_score
import mlflow
from mlflow.tracking import MlflowClient

In [2]:
# 🚩 Config
EXPERIMENT_NAME = "ETF_PEA_MLOpsZoomcamp"
TRACKING_URI = "file:///G:/Mon Drive/DataTalksClub/MLOps-ETF-PEA/mlruns"

DATA_PATH = Path("../data/df_all.parquet")
PREDICTIONS_DIR = Path("../data/predictions")
PREDICTIONS_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
# 🚩 Init MLflow
mlflow.set_tracking_uri(TRACKING_URI)
client = MlflowClient()

In [4]:
# ✅ Load latest model version from Registry
model_name = EXPERIMENT_NAME
latest_versions = client.get_latest_versions(model_name, stages=["None"])
model_uri = f"models:/{model_name}/{latest_versions[0].version}"

model = mlflow.pyfunc.load_model(model_uri)
print(f"✅ Model v{latest_versions[0].version} loaded from MLflow Registry.")

C:\Users\Zelric\AppData\Local\Temp\ipykernel_10872\3383290263.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(model_name, stages=["None"])


✅ Model v1 loaded from MLflow Registry.


In [5]:
# ✅ Load data
df_all = pd.read_parquet(DATA_PATH)
print(f"✅ Data shape: {df_all.shape}")

✅ Data shape: (3802, 32)


In [6]:
# ✅ Predict
feature_cols = [col for col in df_all.columns if col not in ["target", "ticker"]]
X = df_all[feature_cols]
y_true = df_all["target"]

y_pred = model.predict(X)

In [7]:
# ✅ Save predictions
df_all["prediction"] = y_pred
pred_file = PREDICTIONS_DIR / "predictions.parquet"
df_all.to_parquet(pred_file, index=False)
print(f"✅ Predictions saved to {pred_file}")

✅ Predictions saved to ..\data\predictions\predictions.parquet


In [8]:
# ✅ Quick evaluation
rmse = root_mean_squared_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)
print(f"✅ Evaluation on full dataset -> RMSE: {rmse:.5f}, R²: {r2:.5f}")

✅ Evaluation on full dataset -> RMSE: 0.03199, R²: 0.58002


In [9]:
# ✅ Display sample
df_all[["target", "prediction"]].head()

,target,prediction
0,-0.002188,0.006127
1,-0.006561,0.003394
2,0.006937,0.004475
3,-0.007032,0.005034
4,-0.044017,-0.007682
